In [2]:
from google.colab import drive
drive.mount('/content/drive')
# #!ls "/content/drive/My Drive/collab_sandbox"
%cd drive/MyDrive/collab_sandbox/NER/ner_tenses_recognition/active_learning/
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/collab_sandbox/NER/ner_tenses_recognition/active_learning
active_learning_classif_meaningful_sentences


In [26]:
!ls

active_learning
active_learning_classif_meaningful_sentences
advanced_tutorial_crf_lstm.ipynb
best-val-model-86.pt
datasets
index.html
logs
meaningful_classif
med_gen_4_23_21_medium_sentences_dataset.csv
med_gen_4_23_21_medium_sentences_dataset_labeled.csv
medium_articles
medium_sentences_dataset-2-22-21.csv
medium_sentences_dataset-2-22-21.gsheet
medium_sentences_dataset_labeled.csv
medium_sentences_dataset_labeled.gsheet
medium_sentenses_labeled_all__4_23_21_80479.csv
medium_sentenses_labeled_all__4_23_21_80511.csv
my_nerda
NER_A1_tenses.ipynb
NERDA.ipynb
NERDA_models
NER_SANDBOX.ipynb
NER_tenses
NER_tenses_pytorch_lighting_catalyst
ner_tenses_recognition_conll_type.ipynb
NER_tenses_tinkoff
ner-test.csv
results
runs
spacy_ner
spacy_recognition
spacy_textcat
Token_classification.ipynb


https://superintendent.readthedocs.io/en/latest/examples/labelling-text.html

https://www.aclweb.org/anthology/P19-1401.pdf

https://streamlit.io/


# Python widgets

In [20]:
from ipywidgets import GridspecLayout
from IPython.display import clear_output
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import os
import time


class ExampleSentence():
    def __init__(self, 
                 examples=None,
                 output_file=None,
                 dirname='./'
                 ):
      assert output_file != None, "output filename is None"
      # assert not output_file in os.listdir(f'{dirname}'), "File with this name already exists, be careful"
      
      self.output_file = output_file
      self.output_path = None
      self.dirname = dirname

      self.examples = examples
      self.current_elem_position = 0
      self.examples_len = len(examples) - 1
      self.current_value = "yes"
      self.current_example = examples[0]
      
      self.labeled_examples = examples
      self.labels = [self.current_value for _ in examples]
      ### elements
      self.progress = None
      self.toggle_buttons = None
      self.example_text = None
      self.next_button = None
      self.prev_button = None
      self.output = None
      self.grid = GridspecLayout(4, 4, width='800px', height='300px')
      ###
      self.add_elements()
      
      self.setup()
    def setup(self):
      self.toggle_buttons.observe(self.on_value_change, names='value',) 
      self.next_button.on_click(self.next_example)
      self.prev_button.on_click(self.prev_example)

      self.grid[:1, :] =  self.example_text
      self.grid[2, 1:3] = self.toggle_buttons
      self.grid[2, 3:] = self.next_button
      self.grid[2, :1] = self.prev_button
      self.grid[3, :2] = self.progress

      display(self.grid, self.output)

    def prev_example(self, some):
      if self.current_elem_position > 0:
        self.current_elem_position -= 1
        pos = self.current_elem_position
        self.progress.value = pos
        self.toggle_buttons.value = self.labels[pos]
        self.current_example = self.labeled_examples[pos]
        self.set_html_text(self.current_example)

    
    def next_example(self, some):
      if self.current_elem_position < self.examples_len:
        self.labels[self.current_elem_position] = self.current_value
        # print(self.labels)
        self.current_elem_position += 1
        self.progress.value = self.current_elem_position
        self.current_example = self.examples[self.current_elem_position]
        self.toggle_buttons.value = 'yes'
        self.set_html_text(self.current_example)

      elif self.current_elem_position == self.examples_len:
        self.labels[self.current_elem_position - 1] = self.current_value
        self.current_elem_position += 1
        print("End labeling")
        # TODO: some network train stuff
        # create dataframe etc
        self.create_dataframe()

    def create_dataframe(self):
        data = {
            'sentence': self.examples,
            'label': self.labels
        }
        df = pd.DataFrame(data=data)
        timestamp = time.ctime()
        time_uniqueid = time.time()
        self.output_path = f"{self.dirname}{self.output_file}__{timestamp}_{time_uniqueid}.csv"
        df.to_csv(self.output_path, index=False)

    def on_value_change(self, change):
        with self.output:
          self.current_value = change['new']
          

    def set_html_text(self, text):
      render_html = f"""<p style="font-size: 1.3em;padding:16px 32px;">{text}</p>"""
      self.example_text.value = render_html

    def add_elements(self):
      self.progress = widgets.FloatProgress(
              value=0,
              min=0,
              max=self.examples_len,
              description='Examples:',
              bar_style='info',
              style={'bar_color': 'green'},
              orientation='horizontal'
          )
      self.toggle_buttons = widgets.ToggleButtons(
          options=['yes', 'no'],
          description='',
          disabled=False,
          button_style='',
      )
      
      self.example_text = widgets.HTML(value= "")
      self.set_html_text(self.current_example)

      self.next_button = widgets.Button(
          description='Next',
          disabled=False,
          button_style='',
          tooltip='Next',
          icon='check' 
      )
      self.prev_button = widgets.Button(
          description='Prev',
          disabled=False,
          button_style='',
          tooltip='Prev',
          icon='check' 
      )
      self.output = widgets.Output()
sentences_for_labeling = ['The Label widget is useful if you', 'Another test sentence', "Example 3", "Example 4"]
tog = ExampleSentence(examples=sentences_for_labeling,
                      output_file="test_label",
                      dirname='./test_folder/'
                      )

GridspecLayout(children=(HTML(value='<p style="font-size: 1.3em;padding:16px 32px;">The Label widget is useful…

Output()

End labeling


In [ ]:
print(widgets.ToggleButtons.observe.__doc__)

In [ ]:
print(widgets.ToggleButtons.__doc__)

In [ ]:
print(widgets.HTML.observe.__doc__)


# Connect labeler and model

In [24]:
from sklearn.metrics import accuracy_score as sklearn_accuracy

class ClassifModel(nn.Module):
    def __init__(self, transformer, n_tags=1, dropout = 0.1):
      super(ClassifModel, self).__init__()
      transformer_name = transformer.name_or_path
      transformer_config = AutoConfig.from_pretrained(transformer_name)
      self.transformer = transformer
      self.dropout = nn.Dropout(dropout)
      dimension  = transformer_config.hidden_size * 2
      self.output_1 = nn.Linear(dimension, 1)
      # self.output_2 = nn.Linear(dimension, n_tags)
      # self.output_3 = nn.Linear(2, n_tags)
      
      self.batch_norm_1 = torch.nn.BatchNorm1d(transformer_config.hidden_size * 2, )
      # self.batch_norm_2 = torch.nn.BatchNorm1d(transformer_config.hidden_size, )

      for parameter in self.transformer.parameters():
          parameter.requires_grad = False

      # print(self.transformer)
      for i, m in enumerate(self.transformer.encoder.layer):
          if i+1 > 12 - 1:
              for parameter in m.parameters():
                  parameter.requires_grad = True 

    def forward(self, 
                input_ids, 
                attention_mask,  
                ):
        transformer_inputs = {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            }
        outputs = self.transformer(**transformer_inputs, output_hidden_states=True)#[0][:, -1]
        layer_1_cls = outputs[2][-1][:, 0]
        layer_1_sep = outputs[2][-1][:, -1]
        outputs_1 = self.batch_norm_1(torch.cat([layer_1_cls, layer_1_sep], axis=-1))
        
        outputs_1 = self.dropout(outputs_1)
        outputs_1 = self.output_1(outputs_1)
        outputs_1 = outputs_1.squeeze(1)
        return torch.sigmoid(outputs_1)

class ClassifExperiment:
    def __init__(self, 
                 model_name = "roberta-base", 
                 dataset = None,
                 max_len = 128,
                 network = ClassifModel,
                 train_batch_size = 4,
                 dropout = 0.1,
                 grad_clip = 1,
                 optimizer_class = AdamW,
                 classes = 1,
                 target_field = 'Tag',
                 feature_field = 'Text'
                 ):
        self.device = 'cuda' 

        self.model_name = model_name  
        self.dataset = dataset
        self.max_len = max_len
        self.transformer_model = AutoModel.from_pretrained(model_name)
        self.transformer_tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                                                   do_lower_case=True
                                                                   )
        self.transformer_config = AutoConfig.from_pretrained(model_name)  
        self.network = network(self.transformer_model, classes, dropout = dropout)
        self.network.to(self.device)
        self.train_losses = []
        self.valid_loss = np.nan
        self.train_batch_size = train_batch_size
        
        self.dl_train = None
        self.dl_validate = None
        self.dataset_validation = None
        self.target_field = target_field
        self.feature_field = feature_field
        self.train_examples = 100
        self.best_valid_loss = 10
        self.classes = classes

        self.scheduler = None
        self.scheduler_class = get_linear_schedule_with_warmup
        self.optimizer = None
        self.criterion = torch.nn.MSELoss()
        self.clip = grad_clip
        self.optimizer_class = optimizer_class
        self.setup()
        

    def experiment(self, 
                    epochs = 1,
                    warmup_steps = 300,
                    learning_rate = 5e-5
                   ):
        optimizer_parameters = self.network.parameters()

        num_train_steps = int(self.train_examples / self.train_batch_size * epochs)
        
        self.optimizer = self.optimizer_class(optimizer_parameters, lr = learning_rate)
        self.scheduler = self.scheduler_class(
            self.optimizer, num_warmup_steps = warmup_steps, num_training_steps = num_train_steps
        )

        train_losses = []
        best_parameters = None

        for epoch in range(epochs):
            
            print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

            train_loss = self.train()
            train_losses.append(train_loss)
            valid_loss = self.validate()

            print(f"Train Loss = {train_loss} Valid Loss = {valid_loss}")

            if valid_loss < self.best_valid_loss:
                best_parameters = self.network.state_dict()            
                self.best_valid_loss = valid_loss
        print(f"Best val => {self.best_valid_loss}")
        if best_parameters != None:
          self.network.load_state_dict(best_parameters)
        

        return "Model trained successfully"
    def setup(self):
      dataset_training, dataset_validation = train_test_split(self.dataset, test_size=0.25, random_state=42) 
      self.dataset_validation = dataset_validation
      self.train_examples = len(dataset_training)
      self.dl_train = self.create_dataloader(dataset = dataset_training)
      self.dl_validate = self.create_dataloader(dataset = dataset_validation)

    def create_dataloader(self, dataset):

      data_reader = MeaningfullDataset(
          dataset = dataset, 
          tokenizer = self.transformer_tokenizer, 
          )

      data_loader = torch.utils.data.DataLoader(
          data_reader, batch_size = self.train_batch_size, num_workers = 0,
      )
      return data_loader

    def train(self,):
      self.network.train()    
      final_loss = 0.0
      
      for dl in self.dl_train:

          self.optimizer.zero_grad()
          # print(dl)
          outputs = self.network(
              input_ids=dl['input_ids'], 
              attention_mask=dl['attention_mask'],
          )
          loss = self.compute_loss(outputs, 
                              dl.get('label'),
                              dl.get('attention_mask'), 
                              )
          # print(loss)
          loss.backward()
          # torch.nn.utils.clip_grad_norm_(self.network.parameters(), self.clip)
          self.optimizer.step()
          self.scheduler.step()
          final_loss += loss.item()

      return final_loss / len(self.dl_train)
    def validate(self):
        self.network.eval()
        final_loss = 0.0

        for dl in self.dl_validate:
            outputs = self.network(
              input_ids=dl['input_ids'], 
              attention_mask=dl['attention_mask'],
            )
            loss = self.compute_loss(outputs, 
                                dl.get('label'),
                                dl.get('attention_mask'), 
                                )
            final_loss += loss.item()
        
        return final_loss / len(self.dl_validate)   
    def compute_loss(self, preds, target_tags, masks,):
        active_logits = preds.view(-1, self.classes).squeeze(1)
        loss = self.criterion(active_logits, target_tags)
        return loss
    def evaluate_performance(self, dataset=None):
        if dataset is None:
          dataset = self.dataset_validation
        y_true = dataset[self.target_field].tolist()
        y_pred = self.predict(dataset)
        y_pred_1 = []
        for item in y_pred:
          y_pred_1.extend(item)

        f1 = precision_recall_fscore_support(y_true = y_true,
                                             y_pred = y_pred_1,
                                             labels = ['yes', 'no'],
                                             average=None) #'macro'
        # print(f1)
        df = list(zip(['no', 'yes'], f1[2]))
        df = pd.DataFrame(df, columns = ['Level', 'F1-Score'])    

        f1_micro = precision_recall_fscore_support(y_true = y_true,
                                             y_pred = y_pred_1,
                                             labels = ['yes', 'no'],
                                             average='micro') #'micro'
        f1_micro = pd.DataFrame({'Level' : ['AVG_MICRO'], 'F1-Score': [f1_micro[2]]})
        accuracy = pd.DataFrame({'Level': ['acc'], "Accuracy": [sklearn_accuracy(y_true, y_pred_1)]})
        df = df.append(f1_micro)
        df = df.append(accuracy)
      
        return df 
    def predict(self, dataset):
      self.network.eval()
      dl_test = self.create_dataloader(dataset=dataset)

      predictions = []
      labels = ['no', 'yes']
      amount = 0
      with torch.no_grad():
        for i, dl in enumerate(dl_test): 
          outputs = self.network(
              input_ids=dl['input_ids'], 
              attention_mask=dl['attention_mask'],
          )   
          # print(outputs)
          preds = outputs.round().cpu().numpy().tolist()
          preds = [labels[int(item)] for item in preds]
          predictions.append(preds)
      return predictions

# my_classif = ClassifExperiment(dataset=meaningful_sentences, dropout = 0.15)

NameError: ignored

# Streamlit

In [ ]:
!pip install streamlit pyngrok

In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
import pandas as pd

import streamlit as st
import pandas as pd
import numpy as np

st.title('Uber pickups in NYC')

DATE_COLUMN = 'date/time'
DATA_URL = ('https://s3-us-west-2.amazonaws.com/'
            'streamlit-demo-data/uber-raw-data-sep14.csv.gz')

@st.cache
def load_data(nrows):
    data = pd.read_csv(DATA_URL, nrows=nrows)
    lowercase = lambda x: str(x).lower()
    data.rename(lowercase, axis='columns', inplace=True)
    data[DATE_COLUMN] = pd.to_datetime(data[DATE_COLUMN])
    return data

data_load_state = st.text('Loading data...')
data = load_data(10000)
data_load_state.text("Done! (using st.cache)")

if st.checkbox('Show raw data'):
    st.subheader('Raw data')
    st.write(data)

st.subheader('Number of pickups by hour')
hist_values = np.histogram(data[DATE_COLUMN].dt.hour, bins=24, range=(0,24))[0]
st.bar_chart(hist_values)

# Some number in the range 0-23
hour_to_filter = st.slider('hour', 0, 23, 17)
filtered_data = data[data[DATE_COLUMN].dt.hour == hour_to_filter]

st.subheader('Map of all pickups at %s:00' % hour_to_filter)
st.map(filtered_data)

Overwriting app.py


In [ ]:
%%writefile run_streamlit.sh
streamlit run /content/app.py &>/dev/null&
npx localtunnel --port 8501

Writing run_streamlit.sh


In [ ]:
!chmod +x ./run_streamlit.sh

In [ ]:
# !pip install streamlit
# !npm install localtunnel

In [ ]:
# !streamlit run /content/app.py &>/dev/null&
# !npx localtunnel --port 8501

npx: installed 22 in 2.799s
your url is: https://brown-penguin-89.loca.lt
^C


In [ ]:
!pwd 

/content


In [ ]:
/content/app.py

SyntaxError: ignored